# Auto Generated Agent Chat: Group Chat with Coder and Visualization Critic

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
# %%capture --no-stderr
# %pip install pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen

config_list_gpt = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

## Construct Agents

<img src="viz_gc.png" alt="drawing" width="500"/>

In [ ]:
llm_config = {"config_list": config_list_gpt, "seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="人类管理员",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat", "use_docker": False},
   human_input_mode="ALWAYS",
   is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)
coder = autogen.AssistantAgent(
    name="Coder",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""评论员。您是一个非常擅长评估给定可视化代码质量的有益助手，通过提供从 1（差）到 10（好）的评分，并提供清晰的理由。在每个评估中，您必须考虑可视化最佳实践。具体来说，您可以仔细评估代码的以下维度：

错误（bugs）：是否存在错误、逻辑错误、语法错误或拼写错误？代码可能无法编译的原因是什么？应该如何修复？如果存在任何错误，错误分数必须低于 5 分。
数据转换（transformation）：数据是否适当地转换以适应可视化类型？例如，如果需要，数据集是否被适当地过滤、聚合或分组？如果使用日期字段，日期字段是否首先转换为日期对象等？
目标符合度（compliance）：代码在多大程度上符合指定的可视化目标？
可视化类型（type）：考虑最佳实践，可视化类型是否适合数据和意图？是否有一种可视化类型更适合传达见解？如果有更适合的不同可视化类型，分数必须低于 5 分。
数据编码（encoding）：数据是否适当地编码以适应可视化类型？
美学（aesthetics）：可视化的美学是否适合可视化类型和数据？
您必须为上述每个维度提供一个评分。
{bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
请不要提供代码建议。

最后，基于上述评论，建议提供一份具体的行动清单，以帮助编码人员改进代码。
""",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, critic], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [ ]:
user_proxy.initiate_chat(manager, message="从 https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv 下载数据，绘制中、美、日、德、英、法、印七个国家从 2000 年到 2016 年的名义 GDP 折线图（以美元计价），要求折线图有网格线，图例放置在左上角。将图片保存为文件。接受来自 critic 的反馈并改进代码。")